In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from copy import copy
from math import ceil

path1 = "./data/1/"
path2 = "./data/2/"
path3 = "./data/3/"
path4 = "./data/4/"
path5 = "./data/5/"
path6 = "./data/UpToDate/"


  



In [2]:


def fix_dollars(apple2):
    new_open = []
    new_close = []
    new_high = []
    new_low = []
    for i in range(len(apple2['Open'])):
        new_open.append(float(apple2['Open'][i][1:]))
        new_close.append(float(apple2['Close/Last'][i][1:]))
        new_high.append(float(apple2['High'][i][1:]))
        new_low.append(float(apple2['Low'][i][1:]))
    

    apple2.drop('Open',axis=1,inplace=True)
    apple2.drop('Close/Last',axis=1,inplace=True)
    apple2.drop('High',axis=1,inplace=True)
    apple2.drop('Low',axis=1,inplace=True)
    

    apple2['Close'] = new_close
    apple2['Open'] = new_open
    apple2['High'] = new_high
    apple2['Low'] = new_low
    

In [3]:
def fix_date(apple2):
    month = []
    year = []
    week_start = []
    week_end = []
    flag = False
    for i in range(len(apple2['Date'])):
        month.append(int(apple2['Date'][i][0:2]))
        year.append(int(apple2['Date'][i][6:]))
        today = int(apple2['Date'][i][3:5])
        
        if i == 0:
            tomorrow = int(apple2['Date'][i + 1][3:5])
            if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i + 1][3:5]) - 1 == int(apple2['Date'][i + 2][3:5]) - 2 \
                    == int(apple2['Date'][i + 3][3:5]) - 3 == int(apple2['Date'][i + 4][3:5]) - 4:
                week_start.append(True)
                week_end.append(False)
            elif today + 1 == tomorrow:
                week_start.append(False)
                week_end.append(False)
            else:
                week_start.append(False)
                week_end.append(True)
                flag = True
                continue
        else:
            try:
                tomorrow = int(apple2['Date'][i + 1][3:5])
                if today + 1 == tomorrow:  # I should really be cross referencing with a calendar api, not hacking this together
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(False)
                    week_end.append(True)
                    flag = True
                    continue
                if flag:
                    week_start[-1] = True
                    flag = False
            except:
                yesterday = int(apple2['Date'][i - 1][3:5])
                if int(apple2['Date'][i][3:5]) == int(apple2['Date'][i - 1][3:5]) + 1 == int(
                        apple2['Date'][i - 2][3:5]) + 2 \
                        == int(apple2['Date'][i - 3][3:5]) + 3 == int(apple2['Date'][i - 4][3:5]) + 4:
                    week_start.append(False)
                    week_end.append(True)
                elif yesterday + 1 == today:
                    week_start.append(False)
                    week_end.append(False)
                else:
                    week_start.append(True)
                    week_end.append(False)

    apple2.drop('Date', axis=1, inplace=True)

    apple2['Month'] = month
    apple2['Year'] = year
    apple2['WeekStart'] = week_start
    apple2['WeekEnd'] = week_end



In [4]:

def get_targets(df,KF):
    target_percent_inc = []
    target_best_day = []
    target_worst_sell = []
    target_day_high = []
    target_day_low = []
    options = ['t--'+str(i)+'Low' for i in range(1,KF+1)]
    for i in range(len(df['t-0Low'])):
        today = df['t-0High'][i]
        target_day_high.append(today)
        today_worst = df['t-0Low'][i]
        target_day_low.append(today_worst)
        best = np.max([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])
        worst = np.max([df['t--'+str(j)+'High'][i] for j in range(1,KF+1)])
        best_day = np.argmax([df['t--'+str(j)+'Low'][i] for j in range(1,KF+1)])+1
        target_percent_inc.append(round((best-today)*100/today,0))
        target_worst_sell.append(round((worst-today_worst)*100/today,0))
        target_best_day.append(best_day)
    return target_percent_inc, target_best_day, target_worst_sell,target_day_high,target_day_low



In [5]:
def fix_shift(apple2,KP,KF,updated=False):

    names = ['Date','Volume','Close','Open','High','Low'] 
    cases = pd.DataFrame(apple2.values)
    df = pd.concat([cases.shift(i) for i in range(KP-int(updated),-KF-1,-1)],axis=1)
    df.columns = ['t-'+str(i+int(updated))+x for i in range(KP-int(updated),-KF-1,-1) for x in names]
    return df




In [6]:
def fix_normalize(apple2,df):
    for apple_col in list(apple2):
        if apple_col in ['WeekStart','WeekEnd']:
            continue
        MEAN = apple2[apple_col].mean()
        STD = apple2[apple_col].std()
        for df_col in list(df):
            if apple_col in df_col:
                if apple_col =='Month' or apple_col == 'Year':
                    df = df.astype({df_col: 'int32'})
                else:
                    df[df_col]=round((df[df_col]-MEAN)/STD,0)



In [7]:
def drop_future(df,KF):
    for i in range(0,-KF-1,-1):
        for x in ['Month','Year','Volume','Close','Open','High','Low']:
            df.drop('t-'+str(i)+x,axis=1,inplace=True)



In [8]:
def fix_reverse(apple2):
    return apple2.loc[::-1].reset_index(drop=True)


In [9]:
def fix_null(df):
    df.dropna(inplace=True)
    return df.reset_index(drop=True)
    

In [10]:
def get_mult_money_percs(days,multiplier):
    percs = {}
    for period_length in range(2,days+1):
        percs[period_length]=(-1+multiplier**(period_length/250))
    return percs

In [11]:
print(get_mult_money_percs(25,2))

{2: 0.0055605803984681845, 3: 0.008352454894595018, 4: 0.011152080851304147, 5: 0.013959479790029095, 6: 0.016774673291956255, 7: 0.019597682998190535, 8: 0.022428530609922337, 9: 0.025267237888593863, 10: 0.028113826656066543, 11: 0.030968318794788674, 12: 0.0338307362479644, 13: 0.03670110101972068, 14: 0.03957943517527873, 15: 0.042465760841121414, 16: 0.0453601002051649, 17: 0.04826247551692875, 18: 0.05117290908770644, 19: 0.054091423290737906, 20: 0.05701804056138027, 21: 0.05995278339728172, 22: 0.06289567435855314, 23: 0.06584673606794245, 24: 0.06880599121100794, 25: 0.07177346253629313}


In [12]:
def mult_money_op_days(df,days,mult,off_by=0):
    percs = get_mult_money_percs(days,mult)
    wins = []
    for i in range(len(df)):
        win = False
        today_low = df['t-0High'][i]
        for j in percs.keys():
            required_perc = percs[j]
            high = df['t--'+str(j-1)+'High'][i]
            perc_inc = (high-((today_low*(1+off_by))))/today_low
            if perc_inc>=required_perc:
                win = True
                wins.append(1)
                break
        if not win:
            wins.append(0)
    return wins
    

In [13]:
KP = 0
KF = 24
apple = pd.read_csv(path5+"aapl_historical.csv")
#apple = apple.iloc[:200]
apple2 = apple.copy(deep=True)
fix_dollars(apple2)
apple2 = fix_reverse(apple2)
apple2

,Date,Volume,Close,Open,High,Low
0,05/16/2013,602950359,15.5206,15.1157,15.6375,14.9607
1,05/17/2013,427864749,15.4736,15.6804,15.7175,15.3932
2,05/20/2013,450118020,15.8189,15.4254,15.9214,15.3607
3,05/21/2013,455670978,15.7021,15.6482,15.9100,15.5071
4,05/22/2013,442730783,15.7626,15.8589,16.0125,15.6507
...,...,...,...,...,...,...
2512,05/09/2023,45326870,171.7700,173.0500,173.5400,171.6000
2513,05/10/2023,53724500,173.5550,173.0200,174.0300,171.9000
2514,05/11/2023,49514680,173.7500,173.8500,174.5900,172.1700
2515,05/12/2023,45533140,172.5700,173.6200,174.0600,171.0000


In [14]:
#fix_date(apple2)
apple2

df = fix_shift(apple2,KP,KF)
df = fix_null(df)
df




,t-0Date,t-0Volume,t-0Close,t-0Open,t-0High,t-0Low,t--1Date,t--1Volume,t--1Close,t--1Open,...,t--23Close,t--23Open,t--23High,t--23Low,t--24Date,t--24Volume,t--24Close,t--24Open,t--24High,t--24Low
0,05/16/2013,602950359,15.5206,15.1157,15.6375,14.9607,05/17/2013,427864749,15.4736,15.6804,...,15.1071,15.4071,15.4164,15.1071,06/20/2013,357285177,14.8871,14.975,15.2136,14.8275
1,05/17/2013,427864749,15.4736,15.6804,15.7175,15.3932,05/20/2013,450118020,15.8189,15.4254,...,14.8871,14.975,15.2136,14.8275,06/21/2013,480849968,14.7679,14.9461,15.0,14.575
2,05/20/2013,450118020,15.8189,15.4254,15.9214,15.3607,05/21/2013,455670978,15.7021,15.6482,...,14.7679,14.9461,15.0,14.575,06/24/2013,480187768,14.3764,14.55,14.595,14.2161
3,05/21/2013,455670978,15.7021,15.6482,15.91,15.5071,05/22/2013,442730783,15.7626,15.8589,...,14.3764,14.55,14.595,14.2161,06/25/2013,314132994,14.3796,14.4893,14.5639,14.2439
4,05/22/2013,442730783,15.7626,15.8589,16.0125,15.6507,05/23/2013,352959179,15.7907,15.5696,...,14.3796,14.4893,14.5639,14.2439,06/26/2013,367647133,14.2168,14.425,14.4568,14.1307
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,04/04/2023,46278300,165.63,166.595,166.84,165.11,04/05/2023,51511740,163.76,164.74,...,173.5,172.48,173.85,172.11,05/09/2023,45326870,171.77,173.05,173.54,171.6
2489,04/05/2023,51511740,163.76,164.74,165.05,161.8,04/06/2023,45390120,164.66,162.43,...,171.77,173.05,173.54,171.6,05/10/2023,53724500,173.555,173.02,174.03,171.9
2490,04/06/2023,45390120,164.66,162.43,164.9584,162.0,04/10/2023,47716880,162.03,161.42,...,173.555,173.02,174.03,171.9,05/11/2023,49514680,173.75,173.85,174.59,172.17
2491,04/10/2023,47716880,162.03,161.42,162.03,160.08,04/11/2023,47644220,160.8,162.35,...,173.75,173.85,174.59,172.17,05/12/2023,45533140,172.57,173.62,174.06,171.0


In [16]:
wins = mult_money_op_days(df,KF,1.5,.000)
np.average(wins)

0.8182912154031288

target_percent_inc, target_best_day,target_worst_sell,target_day_high,target_day_low = get_targets(df,KF)
np.average(target_day_high)

np.average(target_day_low)

fix_normalize(apple2,df)
df